#  [Split Array Largest Sum](https://leetcode.com/problems/split-array-largest-sum/)
- Split an array into 
    + m non-empty continuous subarrays
    + minimize the largest sum among these m subarrays.

#### Example 1

```
[7,2,5,10,8], m = 2

4 ways of spliting
    [7] [2,5,10,8] largest_sum = 25
    [7,2] [5,10,8] largest_sum = 23
    [7,2,5] [10,8] largest_sum = 18
    [7,2,5,10] [8] largest_sum = 24
    
=> minimum largest_sum = 18
```

#### Example 2

```
[1,2,3,4,5], m = 2

=> minimum largest_sum = 9
[1,2,3] [4,5]
```

#### Example 3

```
[1,4,4], m = 3
=> minimum largest_sum = 4
[1] [4] [4]
```

# Solution

#### Bruteforce $O(n^m)$
- Number of way split n elements into m parts: $C_{n-1}^{m-1}$

<img src="./img/11.jpg" alt="drawing" width="1000"/>


```C++
class Solution {
public:
    vector<int> a;
    int N, M;
    int res;

    void get(int i, int m, int sum_, int max_) {
        if(i == N) {
            if(m == M) res = min(res, max_);
            return;
        }

        // Case: Add a[i] to a new subarray
        if(m<M) 
            get(i + 1, m + 1, a[i], max(max_, a[i]));

        // Case: Add a[i] to current subarray
        if(0<i && i<N)
            get(i + 1, m, sum_ + a[i], max(max_, sum_ + a[i]));
    }

    int splitArray(vector<int>& a, int M) {
        this->a = a;
        this->res = INT_MAX;
        this->N = a.size();
        this->M = M;
  
        get(0, 0, 0, 0);
        return res;
    }
};
```
#### DP Recursive - $O(n^2m)$
- Build prefix sum for O(1) calculation
- Solve the subproblems
<img src="./img/12.jpg" alt="drawing" width="600"/>


```C++
class Solution {
public:
    int N;

    /*--- Prefix Sum ---*/
    vector<int> prefix_sum;
    void build(const vector<int> &a) {
        prefix_sum.assign(a.size()+1, 0);
        int sum_ = 0;
        for(int i=1; i<=a.size(); ++i) {
            prefix_sum[i] = sum_ += a[i-1];
        }
    }
    int query(int i, int j) { // return sum of a in range [i,j]
        return prefix_sum[j+1] - prefix_sum[i];
    }

    /*--- DP ---*/
    // dp[start][m]: Optimal sol in range [start,N-1] with m splits
    vector<vector<int>> dp;
    int get(int start, int m) {
        if(m==1) return query(start, N-1);
        if(dp[start][m] != INT_MAX) return dp[start][m];

        for(int i=start; i<N; ++i) {
            // Prune the search: Array [i+1, N-1] size must >= m-1
            //      Or replace the loop: for(int i=start; i<=N-m; ++i)
            if(N-i-1 >= m-1) dp[start][m] = min(dp[start][m], max(query(start,i), get(i+1,m-1)));
        }


        return dp[start][m];
    }
    int splitArray(vector<int>& nums, int M) {
        // Build prefix_sum
        N = nums.size();
        build(nums);

        // DP
        dp.assign(N, vector<int>(M + 1, INT_MAX));
        return get(0, M);
    }
};
```

#### DP BottomUP - $O(n^2m)$

```C++
class Solution {
public:
    /*--- Prefix Sum ---*/
    vector<int> prefix_sum;
    void build(const vector<int> &a) {
        prefix_sum.assign(a.size()+1, 0);
        int sum_ = 0;
        for(int i=0; i < a.size(); ++i) {
            prefix_sum[i+1] = prefix_sum[i] + a[i];
        }
    }
    int query(int i, int j) { // return sum of a in range [i,j]
        return prefix_sum[j+1] - prefix_sum[i];
    }

    int splitArray(vector<int>& nums, int M) {
        // Build prefix_sum
        int N = nums.size();
        build(nums);

        // Init dp
        vector<vector<int>> dp(N, vector<int>(M + 1, INT_MAX));

        // dp[start][m]: Optimal sol in range [start,N-1] with m splits
        for(int start = N-1; start >= 0; --start) {
            for(int m = 1; m <= M; ++m) {
                if(m==1) {
                    dp[start][m] = query(start, N-1);
                } else {
                    for(int i = N-2; i >= start; --i) {
                        dp[start][m] = min(dp[start][m], max(query(start, i), dp[i+1][m - 1]));
                    }
                }
            }
        }
        return dp[0][M];
    }
};

```


#### BinSearch O(Nlog(N))